In [4]:
 !pip install matplotlib

In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import vgg19
from tensorflow.keras.preprocessing import image as k_image
import matplotlib.pyplot as plt
from IPython.display import display

import os
import random

# Directory path
dir_path = '/workspaces/Week 5/raw_data/monet_jpg'

# List all files in the directory
all_files = [f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))]

# Ensure there are files in the directory
if all_files:
    # Select a random file (image) from the list
    random_image = random.choice(all_files)

    # Provide the path of the randomly selected image
    random_image_path = os.path.join(dir_path, random_image)
    print(random_image_path)
else:
    print("No files found in the specified directory.")

style_reference_image   = random_image_path
base_image_path = '/workspaces/Week 5/raw_data/photo_jpg'

image_files = [f for f in os.listdir(base_image_path) if os.path.isfile(os.path.join(base_image_path, f))]

# Dimensions for resizing images
img_nrows = 256
img_ncols = 256  # Assuming you want a square output, adjust if necessary

# Weights of the different loss components
total_variation_weight = 1e-6
style_weight = 1e-6
content_weight = 2.5e-8

# Create a new figure
fig, axs = plt.subplots(1, 3, figsize=(15, 5))  # Adjust figsize to fit your needs

for i, image_file in enumerate(image_files[:3]):
    image_path = os.path.join(base_image_path, image_file)
    img = k_image.load_img(image_path, target_size=(img_nrows, img_ncols))
    axs[i].imshow(img)
    axs[i].axis('off')

# Show the figure
plt.show()


2023-11-12 14:18:55.942712: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-12 14:18:56.057616: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


/workspaces/Week 5/raw_data/monet_jpg/8314acfd35.jpg


In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

2023-11-10 15:13:40.806405: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-10 15:13:40.857358: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-10 15:13:40.857429: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [ ]:
def preprocess_image(image_input, from_path=True):
    if from_path:
        img = keras.utils.load_img(
            image_input, target_size=(img_nrows, img_ncols)
        )
        img = keras.utils.img_to_array(img)
    else:
        img = tf.image.resize(image_input, (img_nrows, img_ncols))
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return tf.convert_to_tensor(img)



def deprocess_image(x):
    # Util function to convert a tensor into a valid image
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype("uint8")
    return x

# The gram matrix of an image tensor (feature-wise outer product)


def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram


# The "style loss" is designed to maintain
# the style of the reference image in the generated image.
# It is based on the gram matrices (which capture style) of
# feature maps from the style reference image
# and from the generated image


def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels**2) * (size**2))


# An auxiliary loss function
# designed to maintain the "content" of the
# base image in the generated image


def content_loss(base, combination):
    return tf.reduce_sum(tf.square(combination - base))


# The 3rd loss function, total variation loss,
# designed to keep the generated image locally coherent


def total_variation_loss(x):
    a = tf.square(
        x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, 1:, : img_ncols - 1, :]
    )
    b = tf.square(
        x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, : img_nrows - 1, 1:, :]
    )
    return tf.reduce_sum(tf.pow(a + b, 1.25))


# Build a VGG19 model loaded with pre-trained ImageNet weights
model = vgg19.VGG19(weights="imagenet", include_top=False)

# Get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# Set up a model that returns the activation values for every layer in
# VGG19 (as a dict).
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)


# List of layers to use for the style loss.
style_layer_names = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]
# The layer to use for the content loss.
content_layer_name = "block5_conv2"

def compute_loss(combination_image, base_image, style_reference_image):
    
    input_tensor = tf.concat(
        [base_image, style_reference_image, combination_image], axis=0
    )
    features = feature_extractor(input_tensor)

    # Initialize the loss
    loss = tf.zeros(shape=())

    # Add content loss
    layer_features = features[content_layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )
    # Add style loss
    for layer_name in style_layer_names:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        sl = style_loss(style_reference_features, combination_features)
        loss += (style_weight / len(style_layer_names)) * sl

    # Add total variation loss
    loss += total_variation_weight * total_variation_loss(combination_image)
    return loss

@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads


optimizer = keras.optimizers.SGD(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
    )
)

2023-11-10 15:14:09.231816: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-10 15:14:09.231967: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-10 15:14:09.231993: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-10 15:14:09.348030: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-10 15:14:09.348239: I tensorflow/compile

In [ ]:
# Assume that style_reference_image_path is the path to your style reference image
style_reference_image = preprocess_image(style_reference_image)

# Assume that base_image_path is the directory containing your base images
base_image_dir = base_image_path
base_image_files = [f for f in os.listdir(base_image_dir) if os.path.isfile(os.path.join(base_image_dir, f))]

iterations = 100
result_prefix = "output"  # Replace with your desired output file name prefix

for image_file in base_image_files:
    print(f'\nTransforming the image {image_file} ...')
    image_path = os.path.join(base_image_dir, image_file)
    base_image = preprocess_image(image_path)
    combination_image = tf.Variable(preprocess_image(image_path))  
    # Reinitializing the optimizer for the new base image
    optimizer = keras.optimizers.SGD(
        keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
        )
    )

    for i in range(1, iterations + 1):
        loss, grads = compute_loss_and_grads(
            combination_image, base_image, style_reference_image
        )
        optimizer.apply_gradients([(grads, combination_image)])
        if i % 100 == 0:
            print(f"Iteration {i}: loss={loss:.2f}")
            
    # Saving the final version of the image after all iterations
    img = deprocess_image(combination_image.numpy())
    fname = f"/workspaces/Week 5/figures_transformed/{result_prefix}_{image_file}_final.png"
    keras.utils.save_img(fname, img)
    print(f'Figure saved in {fname}')
    tf.keras.backend.clear_session()


NameError: name 'preprocess_image' is not defined

In [ ]:
import os
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi

# Caminho do diretório onde as imagens foram salvas
processed_image_dir = "/workspaces/Week 5/figures_transformed"

# Nome do arquivo zip que será criado
zipfile_name = '/workspaces/Week 5/images.zip'

def zip_images(directory, zip_name):
    # Criando um arquivo zip
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Listando todos os arquivos no diretório
        for root, dirs, files in os.walk(directory):
            for file in files:
                # Adicionando cada arquivo ao zip
                zipf.write(os.path.join(root, file), arcname=file)

# Zipando as imagens
zip_images(processed_image_dir, zipfile_name)

# Submetendo ao Kaggle
api = KaggleApi()
api.authenticate()

# Submeta o arquivo zip à competição
# Substitua 'your-competition-name' pelo nome da competição
# e 'your-submission-message' pela mensagem de submissão desejada
api.competition_submit(zipfile_name, 'my_ first submission', 'gan-getting-started')

print(f'Arquivo {zipfile_name} submetido com sucesso à competição.')


100%|██████████| 768M/768M [00:55<00:00, 14.4MB/s]   


ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json', 'Date': 'Sun, 12 Nov 2023 12:17:17 GMT', 'Access-Control-Allow-Credentials': 'true', 'Set-Cookie': 'ka_sessionid=93b65b4416130e5c329b6d400fbdf655; max-age=2626560; path=/, GCLB=CJDB_9uDhd-AKg; path=/; HttpOnly', 'Vary': 'Accept-Encoding', 'Turbolinks-Location': 'https://www.kaggle.com/api/v1/competitions/submissions/submit/gan-getting-started', 'X-Kaggle-MillisecondsElapsed': '173', 'X-Kaggle-RequestId': '7b842f36502ee9697f79bca93f437dca', 'X-Kaggle-ApiVersion': '1.5.15', 'X-Frame-Options': 'SAMEORIGIN', 'Strict-Transport-Security': 'max-age=63072000; includeSubDomains; preload', 'Content-Security-Policy': "object-src 'none'; script-src 'nonce-dRUMF2gy5HIuihImJgT7fA==' 'report-sample' 'unsafe-inline' 'unsafe-eval' 'strict-dynamic' https: http:; base-uri 'none'; report-uri https://csp.withgoogle.com/csp/kaggle/20201130; frame-src 'self' https://www.kaggleusercontent.com htt